# Example 2: Simple 2D cell signaling model - with curvature-sensitive reactions

We model a reaction between the cell interior and cell membrane in a 2D geometry:
- Cyto - 2D cell "volume"
- PM - 1D cell boundary (represents plasma membrane)

We use the model from [Rangamani et al, 2013, Cell](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3874130/), with a slight variation to illustrate curvature-sensitive reactions. A cytosolic species, "A", reacts with a species on the PM, "X", to form a new species on the PM, "B". We consider four different cases of curvature-sensitivity:
1. No curvature sensitivity (reference case)
2. Binding of A to X preferentially occurs in regions of low curvature.
3. Binding of A to X preferentially occurs in regions of high curvature.
4. X is initially localized to regions of high curvature and cannot diffuse.

In cases 1 and 4, reactions are unaltered from the original model. In cases 2 and 3, reaction definitions include a dependence on curvature. In the original model, A and X binding occurred at rate $k_{on} C_A N_X$, whereas in our altered model, binding occurs at rate $k_{on} \exp(K \text{``curv''}) C_A N_X$. $K$ is a constant that controls the curvature sensitivity (negative for low-curvature biased reaction, positive for high-curvature biased reaction) and $\text{``curv''}$ is the mean curvature computed over the boundary of the mesh.

The resulting PDE and boundary condition for species A are as follows:

$$
\frac{\partial{C_A}}{\partial{t}} = D_A \nabla ^2 C_A \quad \text{in} \; \Omega_{Cyto}\\
\text{B.C. for A:} \quad D_A (\textbf{n} \cdot \nabla C_A)  = -k_{on} \exp(K \text{``curv''}) C_A N_X + k_{off} N_B \quad \text{on} \; \Gamma_{PM}
$$

Similarly, the PDEs for X and B are given by:
$$
\frac{\partial{N_X}}{\partial{t}} = D_X \nabla ^2 N_X - k_{on} \exp(K \text{``curv''}) C_A N_X + k_{off} N_B \quad \text{on} \; \Gamma_{PM}\\
\frac{\partial{N_B}}{\partial{t}} = D_B \nabla ^2 N_B + k_{on} \exp(K \text{``curv''}) C_A N_X - k_{off} N_B \quad \text{on} \; \Gamma_{PM}
$$

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
img_A = mpimg.imread('axb-diagram.png')
plt.imshow(img_A)
plt.axis('off')

Imports and logger initialization:

In [ ]:
import dolfin as d
import sympy as sym
import numpy as np
import pathlib
import logging
import gmsh  # must be imported before pyvista if dolfin is imported first

from smart import config, common, mesh, model, mesh_tools, visualization
from smart.units import unit
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

First, we define the various units for use in the model.

In [ ]:
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
D_unit = um**2 / sec
surf_unit = molecule / um**2
flux_unit = molecule / (um * sec)
edge_unit = molecule / um

Next we generate the model by assembling the compartment, species, parameter, and reaction containers (see Example 1 or API documentation for more details).

In [ ]:
# =============================================================================================
# Compartments
# =============================================================================================
# name, topological dimensionality, length scale units, marker value
Cyto = Compartment("Cyto", 2, um, 1)
PM = Compartment("PM", 1, um, 3)
cc = CompartmentContainer()
cc.add([Cyto, PM])

# =============================================================================================
# Species
# =============================================================================================
# name, initial concentration, concentration units, diffusion, diffusion units, compartment
A = Species("A", 1.0, surf_unit, 10.0, D_unit, "Cyto")
X = Species("X", 1.0, edge_unit, 1.0, D_unit, "PM")
B = Species("B", 0.0, edge_unit, 1.0, D_unit, "PM")
sc = SpeciesContainer()
sc.add([A, X, B])

# =============================================================================================
# Parameters and Reactions
# =============================================================================================

# Reaction of A and X to make B (Cyto-PM reaction)
kon = Parameter("kon", 1.0, 1/(surf_unit*sec))
koff = Parameter("koff", 1.0, 1/sec)
r1 = Reaction("r1", ["A", "X"], ["B"],
              param_map={"on": "kon", "off": "koff"},
              species_map={"A": "A", "X": "X", "B": "B"},
              eqn_f_str=f"A*X*on - B*off")

pc = ParameterContainer()
pc.add([kon, koff])
rc = ReactionContainer()
rc.add([r1])

Now we create a circular mesh (mesh built using gmsh in `smart.mesh_tools`), along with marker functions `mf2` and `mf1`.

In [ ]:
# Create mesh
h_ellipse = 0.1
xrad = 2.0
yrad = 0.5
surf_tag = 1
edge_tag = 3
ellipse_mesh, mf1, mf2 = mesh_tools.create_ellipses(xrad, yrad, hEdge=h_ellipse,
                                                    outer_tag=surf_tag, outer_marker=edge_tag)
# compute curvature at the pm (idenfitied by "edge_tag")
mf_curv = mesh_tools.compute_curvature(ellipse_mesh, mf1, mf2, [edge_tag], [surf_tag])
visualization.plot_dolfin_mesh(ellipse_mesh, mf2, view_xy=True)

Write mesh and meshfunctions to file, then create `mesh.ParentMesh` object.

In [ ]:
mesh_folder = pathlib.Path("ellipse_mesh")
mesh_folder.mkdir(exist_ok=True)
mesh_file = mesh_folder / "ellipse_mesh.h5"
mesh_tools.write_mesh(ellipse_mesh, mf1, mf2, mesh_file)
# save curvatures for reference
curv_file_name = mesh_folder / "curvatures.xdmf"
curv_file = d.XDMFFile(str(curv_file_name))
curv_file.write(mf_curv)

parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_file),
    mesh_filetype="hdf5",
    name="parent_mesh",
    curvature=mf_curv,
)

Initialize config.

In [ ]:
config_cur = config.Config()
config_cur.solver.update(
    {
        "final_t": 5.0,
        "initial_dt": 0.05,
        "time_precision": 6,
    }
)

Here, we compare the solution for different cases of curvature sensitivity vs. curvature insensitivity.

In [ ]:
# folder for saving final mesh images
meshimg_folder = pathlib.Path("mesh_images")
meshimg_folder = meshimg_folder.resolve()
meshimg_folder.mkdir(exist_ok=True)
images = {}

curv_const = [0.0, -0.5, 0.5, np.nan]
label_str = ["No curvature sensitivity", "Low curvature reaction", 
             "High curvature reaction", "Curvature-dependent distribution of X"]
for i in range(len(curv_const)):
    if np.isnan(curv_const[i]):
        r1.eqn_f_str = f"A*X*on - B*off"
        X.initial_condition = "exp(0.5*curv)"
        X.initial_condition_expression = "exp(0.5*curv)"
        X.D = 0.0
    else:
        r1.eqn_f_str = f"A*X*exp({curv_const[i]}*curv)*on - B*off"
    model_cur = model.Model(pc, sc, cc, rc, config_cur, parent_mesh)
    model_cur.initialize()
    results = dict()
    result_folder = pathlib.Path(f"resultsEllipse{i}")
    result_folder.mkdir(exist_ok=True)
    for species_name, species in model_cur.sc.items:
        results[species_name] = d.XDMFFile(
            model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
        )
        results[species_name].parameters["flush_output"] = True
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
    avg_A = [A.initial_condition]
    dx = d.Measure("dx", domain=model_cur.cc['Cyto'].dolfin_mesh)
    volume = d.assemble(1.0*dx)
    while True:
        # Solve the system
        model_cur.monolithic_solve()
        # Save results for post processing
        for species_name, species in model_cur.sc.items:
            results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
        int_val = d.assemble(model_cur.sc['A'].u['u']*dx)
        avg_A.append(int_val / volume)
        # End if we've passed the final time
        if model_cur.t >= model_cur.final_t:
            break
    plt.plot(model_cur.tvec, avg_A, label=label_str[i])
    meshimg_file = meshimg_folder / f"ellipse_mesh{i}.png"
    images[i] = visualization.plot(model_cur.sc["A"].u["u"], view_xy=True, filename=meshimg_file)
plt.legend()

Show final condition for all cases.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 15))
for axi, (idx, image) in zip(ax.flatten(), images.items()):
    axi.imshow(image)
    axi.axis('off')
    axi.set_title(label_str[idx])